In [ ]:
%pip install torchaudio

In [ ]:
import render
import rooms.dataset
import evaluate
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import metrics

In [ ]:
torch.set_default_dtype(torch.float32)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
D_0 = rooms.dataset.dataLoader("classroomBase")

In [ ]:
L_0 = render.get_listener(source_xyz=D_0.speaker_xyz, listener_xyz = D_0.xyzs[0], surfaces=D_0.all_surfaces, 
                                                speed_of_sound=D_0.speed_of_sound,
                                                parallel_surface_pairs=D_0.parallel_surface_pairs,
                                                max_order=D_0.max_order, max_axial_order=D_0.max_axial_order )

In [ ]:
R_0 = render.Renderer(n_surfaces=len(D_0.all_surfaces))

In [ ]:
pt_file = torch.load('./models/classroomBase/weights.pt', map_location=device)
R_0.energy_vector = nn.Parameter(pt_file['model_state_dict']['energy_vector'])
R_0.source_response = nn.Parameter(pt_file['model_state_dict']['source_response'])
R_0.directivity_sphere = nn.Parameter(pt_file['model_state_dict']['directivity_sphere'])
R_0.decay = nn.Parameter(pt_file['model_state_dict']['decay'])
R_0.RIR_residual = nn.Parameter(pt_file['model_state_dict']['RIR_residual'])
R_0.spline_values = nn.Parameter(pt_file['model_state_dict']['spline_values'])

In [ ]:
RIR_by_direction_0 = R_0.render_RIR_by_directions(L_0)

In [ ]:
loss_with_itself = metrics.training_loss_considering_directionality(RIR_by_direction_0, RIR_by_direction_0)
print(loss_with_itself)